In [1]:
import numpy as np
import pandas as pd

In [2]:
# read vcf files
vcf_reader = open('/Users/gcc/Downloads/cadd_copy-gavin.vcf','r')
column_lines = []
info = []
sample_lines = []
for line in vcf_reader.readlines():
    if line.startswith('#CHROM'):
        column_lines = line.strip('\n').strip('#').split('\t')
    elif line.startswith('##INFO'):
        info.append(line.strip('\n').strip('#'))
    elif line[0].isdigit():
        line = line.strip('\n')
        sample_lines.append(line)
        
# parse the column names
column_lines = column_lines[:-1]
info_contents = [line[line.find('INFO=ID=')+len('INFO=ID=')+2:line.find(',')] for line in info[:-1]]
line = info[-1]
info_last = line[line.find("annotations: '")+len("annotations: '")+2:-3].split('|')
columns = column_lines+info_contents+info_last
columns = [col.strip(' ') for col in columns]

In [3]:
columns

['CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'CADD',
 'CADD_SCALED',
 'EXAC_AF',
 'EXAC_AC_HOM',
 'EXAC_AC_HET',
 't_Allele',
 'Gene_Name',
 'Annotation',
 'Putative_impact',
 'Gene_ID',
 'Feature_type',
 'Feature_ID',
 'Transcript_biotype',
 'Rank_total',
 'HGVS_c',
 'HGVS_p',
 'cDNA_position',
 'CDS_position',
 'Protein_position',
 'Distance_to_feature',
 'Errors',
 'Classification',
 'Confidence',
 'Reason']

In [4]:
def cut_str(string,begin,end):
    '''
    cheery pick the begin till end of a string
    '''
    if begin not in string:
        return ''
    begin_ind = string.find(begin)+len(begin)
    cut = string[begin_ind:]
    end_ind = cut.find(end)+len(end)-1
    return cut[:end_ind]

In [92]:
# samples = []
# for line in sample_lines:
#     var_info = line.split('\t')[:-1]
#     if ',' in line:
#         var = line.split('\t')[-1].split(',')
#     else:
#         var=line.split('\t')[-1]
#     for gene in var:
#         if 'MYO5B' in gene:
#             var_gene = []
#             var_gene_infos = gene.split(';')[-1].split('|')
#             var_gene.append(cut_str(gene,'CADD=',';'))
#             var_gene.append(cut_str(gene,'CADD_SCALED=',';'))
#             var_gene.append(cut_str(gene,'EXAC_AF=',';'))
#             var_gene.append(cut_str(gene,'EXAC_AC_HOM=',';'))
#             var_gene.append(cut_str(gene,'EXAC_AC_HET=',';'))
#     samples.append(var_info+var_gene+var_gene_infos)
# for sample in samples:
#     assert len(sample)==31

In [11]:
# parse the samples
samples = []
for line in sample_lines:
    line = line.strip(' ')
    var_info = line.split('\t')[:-1]
    if ',' in line:
        genes = line.split('\t')[-1].split(',')
        for gene in genes:
            if 'MYO5B' in gene:
                myo_info = gene
                var_gene_infos = myo_info.split(';')[-1].split('|')
    else:
        myo_info=line.split('\t')[-1]
        var_gene_infos = myo_info.split(';')[-2].split('|')
    var_gene = []    
    var_gene.append(cut_str(myo_info,'CADD=',';'))
    var_gene.append(cut_str(myo_info,'CADD_SCALED=',';'))
    var_gene.append(cut_str(myo_info,'EXAC_AF=',';'))
    var_gene.append(cut_str(myo_info,'EXAC_AC_HOM=',';'))
    var_gene.append(cut_str(myo_info,'EXAC_AC_HET=',';'))
    samples.append(var_info+var_gene+var_gene_infos)
for sample in samples:
    assert len(sample)==31

In [12]:
datatable = pd.DataFrame(samples,columns=columns)

In [13]:
datatable.replace('',np.NaN)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,CADD,CADD_SCALED,EXAC_AF,...,HGVS_c,HGVS_p,cDNA_position,CDS_position,Protein_position,Distance_to_feature,Errors,Classification,Confidence,Reason
0,18,47352774,.,T,C,.,.,0.334537,6.023,NaN,...,c.*67A>G,NaN,NaN,NaN,NaN,67,NaN,Benign,calibrated,Variant CADD score of 6.023 is less than 8.87 ...
1,18,47361716,.,G,A,.,.,14.849415,48.0,NaN,...,c.5392C>T,p.Gln1798*,5680/9520,5392/5547,1798/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 48.0 is greater than 34....
2,18,47361725,.,G,A,.,.,15.701726,52.0,NaN,...,c.5383C>T,p.Arg1795*,5671/9520,5383/5547,1795/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 52.0 is greater than 34....
3,18,47365526,.,G,A,.,.,14.577744,46.0,NaN,...,c.4840C>T,p.Gln1614*,5128/9520,4840/5547,1614/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 46.0 is greater than 34....
4,18,47365610,.,C,CCA,.,.,9.553096,35.0,NaN,...,c.4755_4756insTG,p.Asp1586fs,5043/9520,4755/5547,1585/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 35.0 is greater than 34....
5,18,47367768,.,AA,GC,.,.,6.332952,29.3,NaN,...,c.4667_4668delTTinsGC,p.Leu1556Arg,4956/9520,4667/5547,1556/1848,NaN,NaN,Pathogenic,genomewide,Variant MAF of 0.0 is rare enough to be potent...
6,18,47369763,.,C,G,.,.,4.451087,24.2,NaN,...,c.4460-1G>C,NaN,NaN,NaN,NaN,NaN,NaN,Pathogenic,genomewide,Variant MAF of 0.0 is rare enough to be potent...
7,18,47373609,.,G,A,.,.,12.325906,39.0,NaN,...,c.4366C>T,p.Gln1456*,4654/9520,4366/5547,1456/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 39.0 is greater than 34....
8,18,47380012,.,A,G,.,.,6.116766,28.3,8.264E-6,...,c.4028T>C,p.Leu1343Pro,4316/9520,4028/5547,1343/1848,NaN,NaN,Pathogenic,genomewide,Variant MAF of 8.264E-6 is rare enough to be p...
9,18,47402080,.,G,A,.,.,12.92118,40.0,NaN,...,c.3514C>T,p.Gln1172*,3802/9520,3514/5547,1172/1848,NaN,NaN,Pathogenic,calibrated,Variant CADD score of 40.0 is greater than 34....


In [14]:
datatable['CADD'] = datatable['CADD'].astype('float')
datatable['CADD_SCALED'] = datatable['CADD_SCALED'].astype('float')
# datatable['EXAC_AF'] = datatable['EXAC_AF'].astype('float')
# datatable['EXAC_AC_HOM'] = datatable['EXAC_AC_HOM'].astype('float')
# datatable['Distance_to_feature'] = datatable['Distance_to_feature'].astype('int')
# datatable['Errors'] = datatable['Errors'].astype('int')

In [15]:
datatable.to_csv('data/gavin_res.csv',sep='\t', index=False, header=True)

In [16]:
datatable.CADD

0       0.334537
1      14.849415
2      15.701726
3      14.577744
4       9.553096
5       6.332952
6       4.451087
7      12.325906
8       6.116766
9      12.921180
10     12.776541
11      0.524487
12     10.692736
13      0.132595
14      2.817738
15     11.752624
16      8.837797
17     -0.336167
18      6.923516
19      7.604902
20      5.482661
21      2.905477
22      3.829816
23      6.318276
24      5.852421
25      5.782332
26      6.073709
27      5.265616
28      7.952294
29      3.928802
         ...    
218     5.635733
219     5.061619
220     6.620644
221     5.910215
222     4.734543
223     1.690176
224     0.071664
225     0.069248
226    -0.153400
227     0.624515
228     0.565037
229    -0.190754
230     1.248168
231    -0.029409
232     0.758393
233     0.089257
234    -0.199414
235     0.421282
236     1.025359
237     0.938279
238    -0.723906
239     0.960552
240     1.128826
241     0.247209
242     1.763419
243     0.797472
244     1.677378
245     0.9788